### **API SADVR - Portrait statistique**  
https://www.cen.umontreal.ca/espacedoc/sadvr/  

Ce NoteBook est destiné à l'extraction et la visualisation de statistiques relatives aux professeur·e·s et à leurs expertises à partir de l'API de la vitrine de la recherche (SADVR). 
Celles-ci seront intégrées en un tableau de bord [PowerBI](https://wiki.umontreal.ca/display/SIE/Power+BI) permettant d'avoir un portrait d'ensemble des données.  

---

In [1]:
import pandas as pd
from SADVR_utils import *
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

**Chargement des données**

In [2]:
data = updateInfoProfs()
data

,idsadvr,prenom,nom,in-memoriam,affiliations,etablissementsAffilies,expertise,recrutementRecherche,sexe,langues,institution,unitesRecherche,paysCode,paysNom,formations,prix,publication,communication
0,in13580,Jurgen,Sygusch,False,"[{'fonction': {'codeSad': '42001', 'nom': 'Pro...",[],"{'secteursRecherche': [{'uid': '6', 'codeLangu...",False,M,[],Université de Montréal,[],CA,Canada,[],"[{'vitrine': [], 'autre': []}]",<ul>\r\n\t<li>Pour consulter les publications ...,NaN
1,in13581,Jacques Y.,Perreault,False,"[{'fonction': {'codeSad': '13001', 'nom': 'Dir...",[],"{'secteursRecherche': [{'uid': '10', 'codeLang...",False,M,"[{'nom': 'Anglais', 'medium': 'Écrit'}, {'nom'...",Université de Montréal,[],CA,Canada,"[{'diplome': 'doctorat IIIème cycle', 'annee':...","[{'vitrine': [], 'autre': []}]",NaN,NaN
2,in13583,Gilles,Bleau,False,[],[{'nom': 'Centre hospitalier de l’Université d...,"{'secteursRecherche': [{'uid': '6', 'codeLangu...",False,M,[],Université de Montréal,"[{'idsadvr': 'ur13910', 'nom': 'Réseau provinc...",CA,Canada,[],"[{'vitrine': [], 'autre': []}]",NaN,NaN
3,in13584,Serge,Rossignol,False,"[{'fonction': {'codeSad': '42032', 'nom': 'Pro...",[],"{'secteursRecherche': [{'uid': '6', 'codeLangu...",False,M,[],Université de Montréal,"[{'idsadvr': 'ur13681', 'nom': 'Groupe de rech...",CA,Canada,[],"[{'vitrine': [{'idsadvr': 'pr13591', 'nom': 'P...",NaN,NaN
4,in13585,Emanuela,Cardia,False,"[{'fonction': {'codeSad': '42001', 'nom': 'Pro...",[],"{'secteursRecherche': [{'uid': '10', 'codeLang...",False,F,[],Université de Montréal,"[{'idsadvr': 'ur13618', 'nom': 'Centre interun...",CA,Canada,[],"[{'vitrine': [], 'autre': []}]","<p>'Market Work, Housework and Childcare: A Ti...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2702,in35947,Adèle,Coriati,False,"[{'fonction': {'codeSad': '52070', 'nom': 'Pro...",[],"{'secteursRecherche': [{'uid': '6', 'codeLangu...",False,F,[],Université de Montréal,[],CA,Canada,[],"[{'vitrine': [], 'autre': []}]",NaN,NaN
2703,in35955,Quoc-Huy,Trinh,False,"[{'fonction': {'codeSad': '42016', 'nom': None...",[],"{'secteursRecherche': [], 'disciplines': [], '...",False,NaN,[],Université de Montréal,[],CA,Canada,[],"[{'vitrine': [], 'autre': []}]",NaN,NaN
2704,in35981,Alex,Ferrone,False,"[{'fonction': {'codeSad': '42011', 'nom': 'Pro...",[],"{'secteursRecherche': [{'uid': '10', 'codeLang...",False,M,"[{'nom': 'Français', 'medium': 'Écrit'}, {'nom...",Université de Montréal,[],CA,Canada,[],"[{'vitrine': [], 'autre': []}]",<p><strong>Livres</strong></p>\r\n\r\n<ul>\r\n...,NaN
2705,in35999,Utsav,Sadana,False,"[{'fonction': {'codeSad': '42011', 'nom': 'Pro...",[],"{'secteursRecherche': [{'uid': '1', 'codeLangu...",False,M,[],Université de Montréal,[],CA,Canada,[],"[{'vitrine': [], 'autre': []}]",NaN,NaN


___
**Statistiques sociodémographiques**

- Genre
- Langues parlées
- Langues écrites
- Date d'obtention du dernier diplôme
- Institution de formation / Pays

In [ ]:
fonctionsProf = pd.read_csv('fonctionsProfs.csv')['codeSad'].tolist()

demographics = data[['idsadvr', 'sexe', 'langues', 'formations', 'affiliations']]
toNormalize = ['langues', 'affiliations', 'formations', 'formations.disciplines', 'formations.institutions']
for c in toNormalize:
    demographics = explodeNormalize(demographics, c)

columns = pd.read_csv('columnsDemographics.csv')['columns'].tolist()
demographics = demographics[[x for x in demographics.columns if x in columns]]


demographics = demographics[demographics['affiliations.fonction.codeSad'].isin(fonctionsProf)]
demographics.to_csv('tables/demographics.csv', index=False)

nbProfs = len(demographics['idsadvr'].drop_duplicates())

Le répertoire de la vitrine de la recherche du SADVR compte actuellement 2613 profils de professeur·e·s.


In [4]:
demographics

,idsadvr,sexe,langues.nom,langues.medium,affiliations.fonction.codeSad,affiliations.fonction.nom,formations.diplome,formations.annee,formations.institutions.nom,formations.institutions.paysNom,formations.institutions.paysCode
0,in13580,M,NaN,NaN,42001,Professeur titulaire,NaN,NaN,NaN,NaN,NaN
1,in13580,M,NaN,NaN,42032,Professeur associé,NaN,NaN,NaN,NaN,NaN
3,in13581,M,Anglais,Écrit,42001,Professeur titulaire,doctorat IIIème cycle,1984,École Pratique des Hautes Études en Sciences S...,France,FR
4,in13581,M,Anglais,Écrit,42001,Professeur titulaire,doctorat IIIème cycle,1984,École Pratique des Hautes Études en Sciences S...,France,FR
6,in13581,M,Français,Écrit,42001,Professeur titulaire,doctorat IIIème cycle,1984,École Pratique des Hautes Études en Sciences S...,France,FR
...,...,...,...,...,...,...,...,...,...,...,...
11774,in35955,NaN,NaN,NaN,42016,None,NaN,NaN,NaN,NaN,NaN
11775,in35981,M,Français,Écrit,42011,Professeur adjoint,NaN,NaN,NaN,NaN,NaN
11776,in35981,M,Français,Oral,42011,Professeur adjoint,NaN,NaN,NaN,NaN,NaN
11777,in35999,M,NaN,NaN,42011,Professeur adjoint,NaN,NaN,NaN,NaN,NaN


*Genre*

In [5]:
mapping = {'M': 'Hommes', 'F': 'Femmes', 'A': 'Autres'}
genre = plotVariable(demographics, 'sexe', mapping=mapping)

px.pie(
    names = genre['labels'],
    values = genre['count'],
    hole=0.5,
    title='Identité de genre',
    width=500,
    color_discrete_sequence= px.colors.qualitative.Antique   
)

*Langues (parlées, écrites)*

In [6]:
# Langues parlées
langueParle = demographics[demographics['langues.medium'] == 'Oral'].drop(columns=['langues.medium'])
langueParle = pd.DataFrame(plotVariable(langueParle, 'langues.nom'))
langueParle = groupOtherValues(langueParle)

px.pie(
    langueParle,
    values = langueParle['count'], 
    names = langueParle['labels'], 
    title='Langues parlées',
    hole=0.5,
    category_orders={'labels': 
        ['Français', 'Anglais', 'Espagnol; castillan', 
         'Allemand', 'Italien', 'Arabe', 'Autre']},
    color_discrete_sequence= px.colors.qualitative.Pastel1,  
    width=600
)

In [7]:
# Langues écrites

langueEcrite = demographics[demographics['langues.medium'] == 'Écrit'].drop(columns=['langues.medium'])
langueEcrite = pd.DataFrame(plotVariable(langueEcrite, 'langues.nom'))
langueEcrite = groupOtherValues(langueEcrite)

order = [x for x in langueEcrite['labels'] if not (x == 'Autre')] + ['Autre']

px.pie(
    langueEcrite,
    values = langueEcrite['count'], 
    names = langueEcrite['labels'], 
    title='Langues écrites',
    hole=0.5,
    category_orders={'labels': order},
    width=600
)

*Nombre de langues parlées*

In [8]:
langues = demographics[demographics['langues.medium'] == 'Oral'][['idsadvr', 'langues.nom', 'sexe']].drop_duplicates()
nbLangues = pd.DataFrame(langues.groupby(['idsadvr', 'sexe'])['langues.nom'].count()).reset_index().sort_values(by='langues.nom', ascending=False)
freqNbLangues = pd.DataFrame(nbLangues['langues.nom'].value_counts()).reset_index()
freqNbLangues

graphLangueEcrite = px.pie(
    freqNbLangues, 
    values='count', 
    names=freqNbLangues['langues.nom'].apply(lambda x: str(x) + " langue(s) parlée(s)"), 
    title='Nombre de langues parlées',
    hole=0.5,
    color = 'langues.nom',
    width=600
)

graphLangueEcrite.show()

*Fonction*

In [9]:
mappingFonction = pd.read_csv('tables/SADVR_fonctions.csv')[['codeSad', 'nomM']].to_dict('records')
mappingFonction = {x['codeSad'] : x['nomM'] for x in mappingFonction}

fonction = pd.DataFrame(plotVariable(demographics, 'affiliations.fonction.codeSad', mapping=mappingFonction))
fonction

,labels,count
0,Professeur titulaire,879
1,Professeur agrégé,427
2,Professeur associé,321
3,Professeur adjoint,272
4,Professeur adjoint de clinique,268
5,Professeur émérite,150
6,Professeur agrégé de clinique,144
7,Chercheur,78
8,Professeur accrédité,65
9,Professeur titulaire de clinique,42


In [10]:
fig = px.bar(
    fonction.sort_values(ascending=True, by='count'), 
    x="count", 
    y="labels", 
    orientation='h',
    title = 'Fonction',
    height=600)

fig.to_html('visualisations/fonctions.html')
fig.show()

In [11]:
fonctionGenre = demographics[['idsadvr', 'sexe', 'affiliations.fonction.codeSad']].drop_duplicates()
freqFonctionGenre = pd.DataFrame(fonctionGenre[['sexe', 'affiliations.fonction.codeSad']].value_counts()).reset_index()

mapping = pd.read_csv('tables/SADVR_fonctions.csv')[['codeSad', 'nomM']].to_dict('records')
mapping = {x['codeSad'] : x['nomM'] for x in mapping}

freqFonctionGenre['fonction'] = freqFonctionGenre['affiliations.fonction.codeSad'].map(mapping)
freqFonctionGenre.to_csv('tables/demographics/fonctionGenre.csv', index=False)

freqFonctionGenre = freqFonctionGenre[['sexe', 'fonction', 'count']]

fig = go.Figure()

# Femmes
dataF = freqFonctionGenre[freqFonctionGenre['sexe'] == 'F'].sort_values(by='count', ascending=True)


fig.add_trace(go.Bar(
    y= dataF['fonction'],
    x= dataF['count'],
    name='Femmes',
    marker_color='violet',
    orientation = 'h',
))

# Hommes
dataH = freqFonctionGenre[freqFonctionGenre['sexe'] == 'M'].sort_values(by='count', ascending=True)

fig.add_trace(go.Bar(
    y= dataH['fonction'],
    x= dataH['count'],
    name='Hommes',
    marker_color='lightblue',
    orientation = 'h'
))

# Autres
# dataA = freqFonctionGenre[freqFonctionGenre['sexe'] == 'A'].sort_values(by='count', ascending=True)

# fig.add_trace(go.Bar(
#     y= dataA['fonction'],
#     x= dataA['count'],
#     name='Autres',
#     marker_color='brown',
#     orientation = 'h'
# ))

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(
    barmode='group', 
    xaxis_tickangle=-45,
    height=900,
    title = 'Distribution des fonctions par genre')

fig.show()

*Lieu de formation*

In [12]:
paysFormation = pd.DataFrame(plotVariable(demographics, 'formations.institutions.paysNom'))
paysFormation = groupOtherValues(paysFormation, 8)
paysFormation

,labels,count
0,Canada,489
1,États-Unis,175
2,France,111
3,Royaume-Uni,38
4,Suisse,15
5,Belgique,11
6,Pays-Bas,11
7,Allemagne,10
0,Autre,60


In [13]:
order = [x for x in paysFormation['labels'] if not (x == 'Autre')] + ['Autre']

px.pie(
    paysFormation, 
    values= paysFormation['count'], 
    names= paysFormation['labels'], 
    title='Formation universitaire Pays',
    hole=0.5,
    width=600, 
    category_orders= {'labels':
        order
    }
)

*Année d'obtention du dernier diplôme*

In [14]:
# D'abord filtrer pour ne conserver que le dernier diplôme obtenu
anneeDiplome = demographics.sort_values(['idsadvr', 'formations.annee'], ascending=[True, False])
anneeDiplome = anneeDiplome[['idsadvr', 'sexe', 'affiliations.fonction.nom', 'formations.annee']].dropna(subset='formations.annee')
anneeDiplome = anneeDiplome.drop_duplicates(subset=['idsadvr', 'formations.annee'])

anneeDiplomeGenre =  pd.DataFrame(anneeDiplome.drop(columns='idsadvr').value_counts()).reset_index().sort_values(by='formations.annee', ascending=True)

anneeDiplome = pd.DataFrame(plotVariable(anneeDiplome, 'formations.annee'))
anneeDiplome = anneeDiplome.sort_values(by='labels', ascending=True)

In [15]:
fig = px.line(
    anneeDiplome, 
    x=anneeDiplome['labels'], 
    y=anneeDiplome['count'],
    title="Année d'obtention du dernier diplôme (Ph.D)")
fig.show()

In [16]:
# D'abord filtrer pour ne conserver que le dernier diplôme obtenu
anneeDiplome = demographics.sort_values(['idsadvr', 'formations.annee'], ascending=[True, False])
anneeDiplome = anneeDiplome[['idsadvr', 'sexe', 'formations.annee']].dropna(subset='formations.annee')
anneeDiplome = anneeDiplome.drop_duplicates(subset=['idsadvr', 'formations.annee'])

anneeDiplomeGenre =  pd.DataFrame(anneeDiplome.drop(columns='idsadvr').value_counts()).reset_index().sort_values(by='formations.annee', ascending=True)
anneeDiplomeGenre

,sexe,formations.annee,count
113,M,1959,1
107,M,1961,2
112,M,1962,1
106,M,1963,2
105,M,1964,2
...,...,...,...
72,F,2020,11
81,F,2021,7
102,M,2021,2
101,F,2022,3


**Expertises de recherche**

- Affiliations (facultés, départements, établissements affiliés)
- Départements
- Disciplines
- Secteur de recherche
- Périodes chronologiques étudiées
- Régions géographiques / pays étudiés

In [17]:
expertises = data[['idsadvr', 'affiliations', 'expertise']]

toNormalize = ['affiliations', 'expertise', 'expertise.secteursRecherche',
                'expertise.disciplines', 'expertise.pays', 
                'expertise.continents', 'expertise.periodesChronologiques']

for c in toNormalize:
    expertises = explodeNormalize(expertises, c)

drop = ['affiliations.courrielInstitutionnel', 'affiliations.immeuble',
        'affiliations.fonction.codeSad', 'affiliations.fonction.nom', 'expertise.motsCles',
       'affiliations.local', 'affiliations.exclusion', 'affiliations.exclusionTel', 'expertise.phraseCle',
       'affiliations.uniteAdministrative.codeSad', 'affiliations.uniteAdministrative.nom',
       'affiliations.telephone.numero', 'affiliations.telephone.poste']

expertises = expertises.drop(columns=drop)
expertises

,idsadvr,affiliations.departement.codeSad,affiliations.departement.nom,affiliations.faculte.codeSad,affiliations.faculte.nom,expertise.secteursRecherche.uid,expertise.secteursRecherche.codeLangue,expertise.secteursRecherche.nom,expertise.secteursRecherche.ordre,expertise.disciplines.uid,...,expertise.pays.codeLangue,expertise.pays.nom,expertise.continents.uid,expertise.continents.ordre,expertise.continents.codeLangue,expertise.continents.nom,expertise.periodesChronologiques.uid,expertise.periodesChronologiques.ordre,expertise.periodesChronologiques.codeLangue,expertise.periodesChronologiques.nom
0,in13580,2310,Département de biochimie et médecine moléculaire,23,Faculté de médecine,6,fre,Sciences de la santé,1,19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,in13580,2310,Département de biochimie et médecine moléculaire,23,Faculté de médecine,6,fre,Sciences de la santé,1,20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,in13580,2310,Département de biochimie et médecine moléculaire,23,Faculté de médecine,6,fre,Sciences de la santé,1,22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,in13580,2310,Département de biochimie et médecine moléculaire,23,Faculté de médecine,6,fre,Sciences de la santé,1,19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,in13580,2310,Département de biochimie et médecine moléculaire,23,Faculté de médecine,6,fre,Sciences de la santé,1,20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284758,in35999,0340,Département d'informatique et de recherche opé...,03,Faculté des arts et des sciences,3,eng,Applied Sciences,1,77,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
284759,in35999,0340,Département d'informatique et de recherche opé...,03,Faculté des arts et des sciences,3,eng,Applied Sciences,1,78,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
284760,in35999,0340,Département d'informatique et de recherche opé...,03,Faculté des arts et des sciences,3,eng,Applied Sciences,1,83,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
284761,in35999,0340,Département d'informatique et de recherche opé...,03,Faculté des arts et des sciences,3,eng,Applied Sciences,1,86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Facultés, départements et établissements affiliés**

Nombre de professeurs par faculté

In [34]:
facultes = pd.DataFrame(plotVariable(expertises, 'affiliations.faculte.nom'))[:-2].sort_values(by='count')
facultes

,labels,count
11,École d'optométrie,31
10,Faculté de médecine dentaire,45
8,Faculté de l'aménagement,65
9,Faculté de musique,65
7,Faculté des sciences infirmières,67
6,Faculté de pharmacie,73
5,Faculté de droit,74
4,Faculté des sciences de l'éducation,79
3,Faculté de médecine vétérinaire,113
2,École de santé publique,174


In [36]:
px.bar(
    paysFormation, 
    x= facultes['count'], 
    y= facultes['labels'], 
    title='Nombre de professeur-e-s par facultés',
    width=1000, 
)

**Expertises de recherche: cartographie des expertises par mots-clés**

- Mots-clés / Phases clés associés aux disciplines / départements de recherche 

Voir si on peut extraire un graphe et le visualiser

**Rayonnement académique**

- Publications
- Communications
- Prix

In [18]:
- Nombre de publications par Faculté


SyntaxError: invalid syntax (2593151634.py, line 1)